# Divergent data reduction for Amor

In this notebook, we will look at how to use the `essreflectometry` package with Sciline, for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

We will begin by importing the modules that are necessary for this notebook.

## Setup

In [ ]:
import scipp as sc
import sciline
from ess import amor
from ess.reflectometry.types import *

In [ ]:
pl = sciline.Pipeline(
    (*amor.providers, *amor.data.providers), params=amor.default_parameters
)


In [ ]:
pl[amor.Chopper1Position[Sample]] = sc.vector(value=[0, 0, -14.842-.500], unit='m')
pl[amor.Chopper2Position[Sample]] = sc.vector(value=[0, 0, -14.842+.500], unit='m')
pl[amor.Chopper1Position[Reference]] = sc.vector(value=[0, 0, -14.842-.500], unit='m')
pl[amor.Chopper2Position[Reference]] = sc.vector(value=[0, 0, -14.842+.500], unit='m')

# Large values disable footprint correction
pl[SampleSize[Sample]] = sc.scalar(10000., unit='mm')
pl[SampleSize[Reference]] = sc.scalar(10000., unit='mm')

pl[amor.ChopperPhase[Sample]] = sc.scalar(-5., unit='deg')
pl[amor.ChopperPhase[Reference]] = sc.scalar(-5, unit='deg')

pl[amor.ChopperFrequency[Sample]] = sc.scalar(8.333, unit='Hz')
pl[amor.ChopperFrequency[Reference]] = sc.scalar(8.333, unit='Hz')

#pl[SampleRotation[Sample]] = sc.scalar(5.05, unit='deg')
#pl[DetectorRotation[Sample]] = sc.scalar(10.21, unit='degree')
#pl[PoochFilename[Sample]] = "amor2023n000611.hdf"
#pl[QBins] = sc.geomspace(dim='Q', start=0.005, stop=0.3, num=400, unit='1/angstrom')

pl[SampleRotation[Sample]] = sc.scalar(0.85, unit='deg')
pl[DetectorRotation[Sample]] = sc.scalar(1.8, unit='degree')
pl[PoochFilename[Sample]] = "amor2023n000608.hdf"
pl[QBins] = sc.geomspace(dim='Q', start=0.005, stop=0.115, num=300, unit='1/angstrom')

pl[SampleRotation[Reference]] = sc.scalar(0.65, unit='deg')
pl[DetectorRotation[Reference]] = sc.scalar(1.4, unit='degree')
pl[PoochFilename[Reference]] = "amor2023n000614.hdf"

pl[NeXusDetectorName[Run]] = 'detector'

pl[WBins] = sc.geomspace('wavelength', 2.8, 12, 300, unit='angstrom')

pl[YIndexLimits] =  sc.scalar(11, unit=None), sc.scalar(41, unit=None)
pl[ZIndexLimits] =  sc.scalar(80, unit=None), sc.scalar(370, unit=None)

In [ ]:
pl.visualize(NormalizedIofQ, graph_attr={'rankdir': 'LR'})

In [ ]:
pl.compute(NormalizedIofQ).hist().plot(norm='log')

## Make a $(\lambda, \theta)$ map
A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins. To achieve this, we request the `ThetaData` from the pipeline. In the graph above we can see that `WavelengthData` is required to compute `ThetaData`, therefore it is also present in `ThetaData` so we don't need to require it separately.

In [ ]:
pl.compute(FootprintCorrectedData[Reference])\
    .hist(
        theta=32*14,
        wavelength=pl.compute(WBins),
    )\
    .plot()

In [ ]:
da = pl.compute(FootprintCorrectedData[Sample])\
    .hist(
        theta=32*14,
        wavelength=pl.compute(WBins),
    )
p = da.plot()
p.canvas.xrange = [0, None]
p.canvas.yrange = [0, None]
# Lines only fitting for sample ...608
p.ax.plot((0, 10), (sc.scalar(0), sc.scalar(0.027)))
p.ax.plot((0, 5.8), (sc.scalar(0), sc.scalar(0.027)))
p

This plot can be used to check if the value of the sample rotation angle $\omega$ is correct. The bright triangles should be pointing back to the origin $\lambda = \theta = 0$.

## Save data

We can save the computed $I(Q)$ to an [ORSO](https://www.reflectometry.org) [.ort](https://github.com/reflectivity/file_format/blob/master/specification.md) file using the [orsopy](https://orsopy.readthedocs.io/en/latest/index.html) package.

First, we need to collect the metadata for that file.
To this end, we build a pipeline with additional providers.
We also insert a parameter to indicate the creator of the processed data.

In [ ]:
from ess.reflectometry import orso
from orsopy import fileio

In [ ]:
for p in (*orso.providers, *amor.orso.providers):
    pl.insert(p)

pl[orso.OrsoCreator] = orso.OrsoCreator(
    fileio.base.Person(
        name='Max Mustermann',
        affiliation='European Spallation Source ERIC',
        contact='max.mustermann@ess.eu',
    )
)

Then, we recompute $I(Q)$ and and combine it with the ORSO metadata:

In [ ]:
iofq_dataset = pl.compute(orso.OrsoIofQDataset)

Unfortunately, some metadata could not be determined automatically.
In particular, we need to specify the sample manually:

In [ ]:
iofq_dataset.info.data_source.sample

In [ ]:
iofq_dataset.info.data_source.sample = fileio.data_source.Sample(
    name='Ni / Ti Multilayer',
    model=fileio.data_source.SampleModel(
        stack='air | (Ni | Ti) * 5 | Si',
    ),
)

And we also add the URL of this notebook to make it easier to reproduce the data:

In [ ]:
iofq_dataset.info.reduction.script = (
    'https://scipp.github.io/essreflectometry/examples/amor.html'
)

To support tracking provenance, we also list the corrections that were done by the workflow and store them in the dataset:

In [ ]:
iofq_dataset.info.reduction.corrections = orso.find_corrections(
    pl.get(orso.OrsoIofQDataset)
)

Finally, we can save the data to a file.
Note that `iofq_dataset` is an [orsopy.fileio.orso.OrsoDataset](https://orsopy.readthedocs.io/en/latest/orsopy.fileio.orso.html#orsopy.fileio.orso.OrsoDataset).

In [ ]:
iofq_dataset.save('amor_reduced_iofq.ort')

Look at the first 50 lines of the file to inspect the metadata:

In [ ]:
!head amor_reduced_iofq.ort -n50